<a href="https://colab.research.google.com/github/CharanSaiManda/A/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#What is BERT?


BERT, short for Bidirectional Encoder Representations from Transformers, is a machine learning (ML) framework for natural language processing. In 2018, Google developed this algorithm to improve contextual understanding of unlabeled text across a broad range of tasks by learning to predict text that might come before and after (bi-directional) other text.

BERT is a model with absolute position embeddings so it’s usually advised to pad the inputs on the right rather than the left.

BERT was trained with the masked language modeling (MLM) and next sentence prediction (NSP) objectives. It is efficient at predicting masked tokens and at NLU in general, but is not optimal for text generation.

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [ ]:
configuration

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

####BERT Tokenizer
Bert works on context based embedding. bert has some special tokens such as [CLS], [SEP], [MASK] ...

cls - Start of sentence

sep - end of sentence

mask - used in masked language modeling to mask words in betweeen.

Bert tokenizer preprocess text with those special tokens.

In [ ]:
#importing necessary modules for BertBasedTokenizantion Masked Language Modelling
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
last_hidden_states

tensor([[[-0.1144,  0.1937,  0.1250,  ..., -0.3827,  0.2107,  0.5407],
         [ 0.5308,  0.3207,  0.3665,  ..., -0.0036,  0.7579,  0.0388],
         [-0.4877,  0.8849,  0.4256,  ..., -0.6976,  0.4458,  0.1231],
         ...,
         [-0.7003, -0.1815,  0.3297,  ..., -0.4838,  0.0680,  0.8901],
         [-1.0355, -0.2567, -0.0317,  ...,  0.3197,  0.3999,  0.1795],
         [ 0.6080,  0.2610, -0.3131,  ...,  0.0311, -0.6283, -0.1994]]],
       grad_fn=<NativeLayerNormBackward0>)

####Masked Language Modelling
Masked Language Modeling is a fill-in-the-blank task, where a model uses the context words surrounding a mask token to try to predict what the masked word should be.

Masked language modeling is a great way to train a language model in a self-supervised setting (without human-annotated labels). Such a model can then be fine-tuned to accomplish various supervised NLP tasks.

Since the words can have different meanings in different places the model needs to learn deep and multiple representations of words.

In [ ]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
inputs = tokenizer("I am going to [MASK] italian tonight.", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

'eat'

####Next Sentence Prediction
Next sentence prediction can be handy at times.
This method can enable model to know whether upcoming sentence is related to current sentence or is it random.

In [ ]:
from transformers import BertForNextSentencePrediction
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
#This is an example where next sentence is random.
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors="pt")
outputs = model(**encoding, labels=torch.LongTensor([1]))
logits = outputs.logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
if logits[0, 0] < logits[0, 1]:
  print("Next sentence is random")
else:
  print("Next sentence is not random")

Next sentence is random


####Text classification
BERT models help machines understand and interpret the meaning of the text.



In [ ]:
import pandas as pd
df =pd.read_csv('SMSSpamCollection', sep='\t', names = ["Category","Message"])

In [ ]:
df['Category'] = df.Category.apply(lambda x : 1 if x=='spam' else 0)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow_text as tf_text

BERT will then convert a given sentence into an embedding vector. Embedding vector is used to represent the unique words in a given document. BERT ensures words with the same meaning will have a similar representation.

The BERT process undergoes two stages: Preprocessing and encoding.


BERT models are usually pre-trained. They are available in TensorFlow Hub.

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(df.Message, df.Category, epochs=10, batch_size = 32)

Epoch 1/10
175/175 [==============================] - 66s 315ms/step - loss: 0.2852 - accuracy: 0.8785
Epoch 2/10
175/175 [==============================] - 59s 335ms/step - loss: 0.1997 - accuracy: 0.9200
Epoch 3/10
175/175 [==============================] - 58s 333ms/step - loss: 0.1659 - accuracy: 0.9393
Epoch 4/10
175/175 [==============================] - 58s 331ms/step - loss: 0.1460 - accuracy: 0.9480
Epoch 5/10
175/175 [==============================] - 59s 334ms/step - loss: 0.1297 - accuracy: 0.9575
Epoch 6/10
175/175 [==============================] - 58s 334ms/step - loss: 0.1214 - accuracy: 0.9620
Epoch 7/10
175/175 [==============================] - 58s 331ms/step - loss: 0.1100 - accuracy: 0.9664
Epoch 8/10
175/175 [==============================] - 58s 333ms/step - loss: 0.1045 - accuracy: 0.9690
Epoch 9/10
175/175 [==============================] - 58s 331ms/step - loss: 0.1021 - accuracy: 0.9706
Epoch 10/10
175/175 [==============================] - 58s 332ms/step - l

In [ ]:
df.Message[2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [ ]:
# 0 for not spam and 1 for spam
if model.predict([df.Message[2]]) > 0.5:
  print(1)
else:
  print(0)

1/1 [==============================] - 0s 40ms/step
1
